In [9]:
# Import the onnxruntime_genai library, which is likely used for running generative AI models with ONNX Runtime
import onnxruntime_genai as og

# Import the argparse library to handle command-line arguments
import argparse

# Import the time library to work with time-related functions
import time

In [10]:
# Create an instance of the Model class from the onnxruntime_genai library
# The model is loaded from the specified path, which points to a Phi-3-mini-128k-instruct-onnx model
# The model is optimized for CPU and mobile devices with specific configurations (int4 precision, rtn block size 32, accuracy level 4)
model = og.Model('../../Models/Phi-3-mini-128k-instruct-onnx/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')


In [11]:
# Create an instance of the Tokenizer class from the onnxruntime_genai library
# The tokenizer is initialized with the previously loaded model
tokenizer = og.Tokenizer(model)

# Create a stream for tokenizing input data using the tokenizer instance
tokenizer_stream = tokenizer.create_stream()

In [12]:
# Define a dictionary named search_options to store configuration settings for the model's search behavior
search_options = {
    # Set the maximum length of the generated output to 1024 tokens
    "max_length": 1024,
    
    # Set the temperature parameter to 0.6, which controls the randomness of the output
    # Lower values make the output more deterministic, while higher values make it more random
    "temperature": 0.6
}

In [13]:
# Create an instance of the GeneratorParams class from the onnxruntime_genai library
# The parameters are initialized with the previously loaded model
params = og.GeneratorParams(model)

# Optionally, enable the use of CUDA graphs for optimized performance with a maximum batch size of 1
# This line is currently commented out
# params.try_use_cuda_graph_with_max_batch_size(1)

# Set the search options for the generator parameters using the previously defined search_options dictionary
params.set_search_options(**search_options)

In [14]:
# Define a prompt string that sets up a conversation between the user and the AI assistant
# The prompt includes special tokens to indicate the roles of the system, user, and assistant
prompt = "<|system|>You are a helpful AI assistant.<|end|><|user|>Can you introduce yourself?<|end|><|assistant|>"

# Encode the prompt string into input tokens using the tokenizer
input_tokens = tokenizer.encode(prompt)

# Set the input_ids attribute of the params object to the encoded input tokens
params.input_ids = input_tokens


In [15]:
# Create an instance of the Generator class from the onnxruntime_genai library
# The generator is initialized with the previously loaded model and the configured parameters
generator = og.Generator(model, params)

In [16]:
# Loop until the generator has finished generating tokens
while not generator.is_done():
    # Compute the logits (probabilities) for the next token
    generator.compute_logits()
    
    # Generate the next token based on the computed logits
    generator.generate_next_token()

    # Retrieve the newly generated token
    new_token = generator.get_next_tokens()[0]
    
    # Decode the new token into a readable string and print it
    # The 'end' parameter ensures the output is printed on the same line
    # The 'flush' parameter forces the output to be printed immediately
    print(tokenizer_stream.decode(new_token), end='', flush=True)


 Hello! I'm Phi, an AI developed by Microsoft, designed to assist users by providing accurate information, answering questions, and helping with a variety of tasks. I'm committed to ensuring your experience is as helpful and efficient as possible.